# Robotic Systems 1 - Homework 2

In [ ]:
import numpy as np # Linear Algebra
import pinocchio as pin # Pinocchio library
import os
import time
import helpers as hp

from pinocchio.robot_wrapper import RobotWrapper
from pinocchio.visualize import MeshcatVisualizer

VISUALIZER = MeshcatVisualizer

In [ ]:
def load_franka():
  current_path = os.path.abspath('') # where the folder `robot` is located at
  robot_path = os.path.join(current_path, "robot")

  robot = RobotWrapper.BuildFromURDF(os.path.join(robot_path, "franka.urdf"), package_dirs = robot_path)

  return robot

In [ ]:
def step_world(model, data, t_q, t_qdot, t_tau, t_dt):
  
  qdotdot = pin.aba(model, data, t_q, t_qdot, t_tau)
  qdot = t_qdot + qdotdot*t_dt
  qdot = np.clip(qdot, -model.velocityLimit, model.velocityLimit)
  q = pin.integrate(model, t_q, qdot*t_dt)
  q = np.clip(q, model.lowerPositionLimit, model.upperPositionLimit)
  
  return q, qdot

In [ ]:
robot = load_franka()

model = robot.model
data = robot.data
end_effector = model.getFrameId("panda_ee")

robot.setVisualizer(VISUALIZER())
robot.initViewer()
robot.loadViewerModel("pinocchio")

robot.display(pin.randomConfiguration(model))

## Task-Space Trajectories

I should implement 4 different profiles. I decided to do them in 4 different ways. So we have:
- Screw Path
- Decoupled trajectory
- The above with third order polynomials for time scaling


Questions:
- orientation and translation error should be computed separately in the controller -> straight lines in the task space (not on the SO3 manifold) but what if the trajectory is computed on the manifold? (screw path(on the SO3 manifold) and decoupled rot and trans)

In [ ]:
# def get_Twd(start, goal, t_t, T):
#   return pin.SE3.Interpolate(start, goal, min(1, t_t/T))

# def get_Twd(start, goal, t_t, T):
#   Twd = pin.SE3.Interpolate(start, goal, min(1, t_t/T))
#   Twd.translation = interpolate_translation(start.translation, goal.translation, min(1,t_t/T))
#   return Twd

def get_Twd(start, goal, t_t, T):
  s = 3*(t_t/T)**2 -2*(t_t/T)**3
  Twd = pin.SE3.Interpolate(start, goal, s)
  Twd.translation = hp.interpolate_translation(start.translation, goal.translation, s)
  return Twd

# def get_Twd(start, goal, t_t, T):
#   s = 3*(t_t/T)**2 -2*(t_t/T)**3
#   # return pin.SE3.Interpolate(start, goal, min(1, time/T))
#   return pin.SE3.Interpolate(start, goal, s)

# def get_Twd(start, goal, t_t, T):
#   s = 3*(t_t/T)**2 -2*(t_t/T)**3
#   # return pin.SE3.Interpolate(start, goal, min(1, time/T))
#   return pin.SE3.Interpolate(start, goal, s)

## Task-Space Controller

In [ ]:
def compute_error(Td, Tb):
  error = pin.log(Td * pin.SE3.inverse(Tb))
  error.linear = Td.translation - Tb.translation

  return error.vector  

In [ ]:



prev_error = None
sum_error = 0.
init = True



def controller(model, data, t_q, t_Twd, t_dt, Kp=100., Ki=1000., Kd= 100.):
    global prev_error
    global sum_error
    global init

    M = pin.crba(model, data, t_q)

    hp.fk_all(model, data, t_q)

    T_wb = data.oMf[end_effector].copy()

    error = compute_error(t_Twd, T_wb)

    if init:
        prev_error = np.copy(error)
        init=False

    sum_error += (error * t_dt)
    
    J = pin.computeFrameJacobian(model, data, t_q, end_effector, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)
    # dJ = pin.computeFrameJacobianTimeVariation(model, data, t_q, end_effector, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)
    J_Tpinv = hp.damped_pseudoinverse(J.T)


    L = J_Tpinv @ M @ hp.damped_pseudoinverse(J)


    F_w = L@(Kp*error + Ki*sum_error + Kd*(error-prev_error)/t_dt)
    prev_error = np.copy(error)

    #Null space controller - regularization
    q_target = (model.upperPositionLimit - model.lowerPositionLimit) / 2. + model.lowerPositionLimit
    t_reg = 100. * (q_target - t_q)
    

    tau = J.T @ F_w  + (np.eye(model.nv) - J.T @ J_Tpinv) @ t_reg


    return tau
  
  

## Simulation

In [ ]:
T = 10.
dt = 0.001
K = int(T/dt) + 1
prev_error = None
sum_error = 0.
init = True

pin.seed(2)
q = pin.randomConfiguration(model)
# q = pin.neutral(model)
# q = (model.upperPositionLimit - model.lowerPositionLimit) / 2. + model.lowerPositionLimit
hp.fk_all(model,data,q)
T_start = data.oMf[end_effector].copy()
qdot = np.zeros(model.nv)
tau = np.zeros(model.nv)



T_end = T_start.copy()
T_end.translation = T_start.translation + [0.,+0.,0.]
T_end.rotation = hp.RotX(-np.pi/2) @ T_start.rotation

robot.display(q)
time.sleep(1)

tau = pin.computeGeneralizedGravity(model,data,q)



t = 0
k=0

for k in range(K):
  Twd = get_Twd(T_start, T_end, t, T)

  tau = controller(model, data, q, Twd, dt, 10000., 0., 1000.)# + pin.computeGeneralizedGravity(model,data,q)

  q , q_dot  = step_world(model, data, q, qdot, tau, dt)

  if np.mod(k, 100)==0:
    robot.display(q)

  t += dt
  k +=1
  
print(compute_error(T_end, data.oMf[end_effector]))




Simple algorithm